In [246]:
# Library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from util.bands import *
from util.pairs_selection import *
from util.backtest import *
import matplotlib.ticker as ticker
plt.rcParams["font.family"] = "NanumGothic"
import warnings
warnings.filterwarnings(action='ignore')

# 페어 선정 없는 버전

# Data load
pairs_list = [#('CJ', 'CJ4우(전환)'),
 ('CJ', 'CJ우'),
 ('CJ제일제당', 'CJ제일제당 우'),
 ('DB하이텍', 'DB하이텍1우'),
 #('DL이앤씨', 'DL이앤씨2우(전환)'),
 #('DL이앤씨', 'DL이앤씨우'),
 ('GS', 'GS우'),
 #('JW중외제약', 'JW중외제약2우B'),
 ('JW중외제약', 'JW중외제약우'),
 ('LG', 'LG우'),
 ('LG생활건강', 'LG생활건강우'),
 ('LG전자', 'LG전자우'),
 ('LG화학', 'LG화학우'),
 #('LX하우시스', 'LX하우시스우'),
 ('NH투자증권', 'NH투자증권우'),
 #('SK', 'SK우'),
 ('SK네트웍스', 'SK네트웍스우'),
 ('SK이노베이션', 'SK이노베이션우'),
 ('S-Oil', 'S-Oil우'),
 ('금호석유', '금호석유우'),
 ('넥센타이어', '넥센타이어1우B'),
 ('대상', '대상우'),
 ('대한항공', '대한항공우'),
 #('미래에셋증권', '미래에셋증권2우B'),
 ('미래에셋증권', '미래에셋증권우'),
 ('삼성SDI', '삼성SDI우'),
 #('삼성물산', '삼성물산우B'),
 ('삼성전기', '삼성전기우'),
 ('삼성전자', '삼성전자우'),
 ('삼성화재', '삼성화재우'),
 #('아모레G', '아모레G3우(전환)'),
 ('아모레G', '아모레G우'),
 ('아모레퍼시픽', '아모레퍼시픽우'),
 ('유한양행', '유한양행우'),
 #('하이트진로', '하이트진로2우B'),
 ('한국금융지주', '한국금융지주우'),
 #('한화', '한화3우B'),
 ('한화', '한화우'),
 ('한화솔루션', '한화솔루션우'),
 ('현대건설', '현대건설우'),
 #('현대차', '현대차2우B'),
 #('현대차', '현대차3우B'),
 ('현대차', '현대차우'),
 ('호텔신라', '호텔신라우')]

In [397]:
pairs_list = [('CJ', 'CJ우'),
 ('LG', 'LG우'),
 ('LG생활건강', 'LG생활건강우'),
 ('LG전자', 'LG전자우'),
 ('LG화학', 'LG화학우'),
 ('NH투자증권', 'NH투자증권우'),
 ('SK이노베이션', 'SK이노베이션우'),
 ('S-Oil', 'S-Oil우'),
 ('미래에셋증권', '미래에셋증권우'),
 ('삼성SDI', '삼성SDI우'),
 ('삼성전자', '삼성전자우'),
 ('삼성화재', '삼성화재우'),
 ('아모레퍼시픽', '아모레퍼시픽우'),
 ('한국금융지주', '한국금융지주우'),
 ('현대차', '현대차우')]

In [345]:
main_path = '/home/lululalamoon/CHO/chosta/data/k_stocks/daily/ohlcv'
ohlcv_list = []
price_pairs = []
stock_names = []
for pair in pairs_list:
    for j in pair:
        data = pd.read_csv(main_path + f'/{j}_ohlcv.csv', index_col=0, parse_dates=True)
        ohlcv_list.append(data)
        stock_names.append(j)
ohlcv_dict = dict(zip(stock_names, ohlcv_list))

all_transactions = []
all_inventories = []
opt_window_list = []
max_pnl_list = []

for i in range(len(pairs_list)):

    common = pairs_list[i][0]
    preferred = pairs_list[i][1]

    common_price = ohlcv_dict[common]['종가'].rename(common)  # 수정종가 사용
    preferred_price = ohlcv_dict[preferred]['종가'].rename(preferred)
    pairs_prices = pd.concat([common_price, preferred_price], axis=1)

    # get spread
    first = pairs_prices.dropna().iloc[:, 0]  # 보통주
    second = pairs_prices.dropna().iloc[:, 1]  # 우선주
    spread = get_log_spread(first, second)  # lny-lnx

    last_invens = []
    windows = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
    for win in windows:
        # make bands
        window = win
        mult = 2
        mult2 = 2.5
        df_bb = bollinger_bands_double(spread, window, mult, mult2).dropna()

        sp, lb, ub, ma = df_bb.price, df_bb.lb, df_bb.ub, df_bb.MA
        lb2, ub2 = df_bb.lb2, df_bb.ub2

        # trade_dates = get_trade_dates_bbd_m(sp, ub, lb, ub2, lb2, ma)
        # trade_dates = get_trade_dates_bbd_m_long(sp, ub, lb, ub2, lb2, ma)
        # trade_dates = get_trade_dates_bb_m_long(sp, lb, ma)
        # trade_dates = get_trade_dates_bb_long(sp, ub, lb)
        trade_dates = get_trade_dates_bbd_long(sp, ub, lb, ub2, lb2)

        initial_invest = 100_000_000
        c = 0.002  # 거래당
        transactions_list = []

        inventory_now = initial_invest
        inventory_history = [inventory_now]

        trade_dates_is = trade_dates.loc[(trade_dates.entry > '2009-1-1') & (trade_dates.entry < '2017-1-1')] # in-sample

        for t in trade_dates_is.index:  # t는 하나의 round-trip
            entry = pd.to_datetime(trade_dates_is.entry[t])
            exit = pd.to_datetime(trade_dates_is.exit[t])

            pos = trade_dates_is.position[t]

            p1_in = first[entry]
            p2_in = second[entry]
            p1_out = first[exit]
            p2_out = second[exit]
            q1 = int(inventory_now / (2 * p1_in))
            q2 = int(inventory_now / (2 * p2_in))

            one_transaction = [[entry, -pos * q1, p1_in, common],
                               [entry, pos * q2, p2_in, preferred],
                               [exit, pos * q1, p1_out, common],
                               [exit, -pos * q2, p2_out, preferred]]
            one_transaction = pd.DataFrame(one_transaction, columns=['index', 'amount', 'price', 'symbol']).set_index(
                'index')

            pnl1_ = (-pos) * (q1 * p1_out - q1 * p1_in)
            cost1_ = c * (q1 * p1_out + q1 * p1_in)
            pnl2_ = pos * (q2 * p2_out - q2 * p2_in)
            cost2_ = c * (q2 * p2_out + q2 * p2_in)

            pnl1 = pnl1_ - cost1_
            pnl2 = pnl2_ - cost2_

            inventory_now += (pnl1 + pnl2)
            inventory_history.append(inventory_now)
            transactions_list.append(one_transaction)

        transactions = pd.concat(transactions_list)

        ind_ = [list(trade_dates_is.entry)[0]] + list(trade_dates_is.exit)
        inventory = pd.Series(inventory_history, index=pd.to_datetime(ind_))
        inventory = inventory.resample('D').last().ffill()
        inventory = inventory.to_frame().rename(
            columns={0: common + ' & ' + preferred})

        if np.any(inventory <= 0):
            from_ind_ = inventory[inventory.iloc[:, 0] <= 0].index[0]
            inventory.loc[from_ind_:, :] = 0

        last_invens.append(inventory.iloc[-1, 0])
    opt_window = windows[np.argmax(np.array(last_invens))]
    opt_window_list.append(opt_window)
    
    all_inventories.append(inventory)
    max_pnl = np.max(np.array(last_invens))
    max_pnl_list.append(max_pnl)
    
    print(common, '&', preferred, "'s Optimal window: ", opt_window)
    

LG생활건강 & LG생활건강우 's Optimal window:  10
LG전자 & LG전자우 's Optimal window:  10
LG화학 & LG화학우 's Optimal window:  20
NH투자증권 & NH투자증권우 's Optimal window:  50
SK이노베이션 & SK이노베이션우 's Optimal window:  20
S-Oil & S-Oil우 's Optimal window:  60
미래에셋증권 & 미래에셋증권우 's Optimal window:  10
삼성SDI & 삼성SDI우 's Optimal window:  60
삼성전자 & 삼성전자우 's Optimal window:  10
삼성화재 & 삼성화재우 's Optimal window:  10
아모레퍼시픽 & 아모레퍼시픽우 's Optimal window:  25
한국금융지주 & 한국금융지주우 's Optimal window:  15
현대차 & 현대차우 's Optimal window:  30


In [353]:
rets = pd.concat(all_inventories,axis=1).pct_change().replace(0,np.nan)

In [369]:
rets.iloc[:,i].dropna().resample('Y').count().mean()

1.75

In [380]:
srdf = pd.DataFrame(index=['ASR'])
for i in range(len(rets.T)):
    sr = (rets.iloc[:,i].dropna().mean() / rets.iloc[:,i].dropna().std()) * np.sqrt(rets.iloc[:,i].dropna().resample('Y').count().mean())
    srdf[rets.columns[i]] = sr

In [279]:
opt_dict = dict(zip(pairs_list, opt_window_list))
maxpnl_dict = dict(zip(pairs_list, max_pnl_list))

In [287]:
df = pd.DataFrame(opt_dict, index=['optimal windows']).T

In [288]:
commons = [df.index[i][0] for i in range(len(df))]
preferreds = [df.index[i][1] for i in range(len(df))]

In [289]:
# 가격 변동성 이용
보통주변동성 = np.array([ohlcv_dict[i].종가['2009-1':'2016-12'].pct_change().std() for i in commons])
우선주변동성 = np.array([ohlcv_dict[i].종가['2009-1':'2016-12'].pct_change().std() for i in preferreds])

df['보통주 수익률 변동성'] = 보통주변동성.round(4)
df['우선주 수익률 변동성'] = 우선주변동성.round(4)

In [290]:
# 거래량 이용

보통주거래량 = np.array([ohlcv_dict[i].거래량['2009-1':'2016-12'].mean() for i in commons])
우선주거래량 = np.array([ohlcv_dict[i].거래량['2009-1':'2016-12'].mean() for i in preferreds])

df['보통주 평균 거래량'] = 보통주거래량.round(2)
df['우선주 평균 거래량'] = 우선주거래량.round(2)

In [284]:
from pykrx import stock
tickers = stock.get_market_ticker_list()
names = []
for ticker in tickers:
    name = stock.get_market_ticker_name(ticker)
    names.append(name)
ticker_dict = dict(zip(names, tickers))

In [285]:
common_tickers = [ticker_dict[i] for i in commons]
preferred_tickers = [ticker_dict[i] for i in preferreds]

In [293]:
mc = stock.get_market_cap("20140103")
보통주시가총액 = np.array(mc['시가총액'].loc[common_tickers])
우선주시가총액 = np.array(mc['시가총액'].loc[preferred_tickers])

df['보통주 시가총액'] = 보통주시가총액
df['우선주 시가총액'] = 우선주시가총액

In [387]:
df['PNL cumsum'] = np.array(max_pnl_list).round() - 100_000_000

In [388]:
df['SR'] = srdf.T.ASR.values

In [392]:
df.sort_values('SR')

,,optimal windows,보통주 수익률 변동성,우선주 수익률 변동성,보통주 평균 거래량,우선주 평균 거래량,보통주 시가총액,우선주 시가총액,보통주평균DPS,우선주평균DPS,PNL cumsum,SR
금호석유,금호석유우,15,0.0300,0.0236,231870.55,12694.69,2787793726500,102647349700,1362.15,1412.36,-16094385.0,-0.335709
GS,GS우,50,0.0216,0.0137,456991.69,4512.36,5175386554600,51135264900,1166.10,1216.05,20034474.0,-0.178686
CJ제일제당,CJ제일제당 우,10,0.0215,0.0173,67577.96,3967.44,3648349443000,123185782400,2289.17,2338.80,21296418.0,-0.126260
아모레G,아모레G우,15,0.0230,0.0262,39495.87,3016.25,3650437335000,125009138000,1581.67,1621.94,85025629.0,-0.069239
삼성전기,삼성전기우,30,0.0225,0.0207,721805.30,15282.59,5191211872000,90261853200,717.88,767.76,608860.0,-0.061623
LG,LG우,20,0.0207,0.0174,484461.11,4607.15,10646774982700,123471718250,983.15,1033.15,34431767.0,-0.041109
CJ,CJ우,10,0.0236,0.0192,139731.17,7513.45,3397746924000,90973975750,939.30,989.30,48326055.0,-0.004039
넥센타이어,넥센타이어1우B,55,0.0267,0.0249,565169.80,25687.19,1371597757000,25545000000,55.99,60.99,53632556.0,0.027208
S-Oil,S-Oil우,60,0.0233,0.0131,379991.36,8408.85,8004636511200,205922662400,2995.81,3020.82,-293169.0,0.031363
대상,대상우,40,0.0272,0.0241,270720.85,7224.60,1273228275000,14113296900,191.43,201.35,20179588.0,0.066261


In [390]:
df_rank = df.iloc[:,0].to_frame().join(df.iloc[:,1:].rank(method='min', ascending=False)).sort_values('optimal windows')

In [391]:
df_rank.sort_values('우선주 시가총액')

,,optimal windows,보통주 수익률 변동성,우선주 수익률 변동성,보통주 평균 거래량,우선주 평균 거래량,보통주 시가총액,우선주 시가총액,보통주평균DPS,우선주평균DPS,PNL cumsum,SR
삼성전자,삼성전자우,10,31.0,22.0,20.0,5.0,1.0,1.0,1.0,1.0,12.0,4.0
현대차,현대차우,30,23.0,13.0,6.0,1.0,2.0,2.0,9.0,9.0,11.0,19.0
LG화학,LG화학우,20,14.0,12.0,19.0,6.0,3.0,3.0,3.0,3.0,8.0,11.0
삼성화재,삼성화재우,10,30.0,28.0,27.0,26.0,5.0,4.0,4.0,4.0,16.0,7.0
LG생활건강,LG생활건강우,10,22.0,17.0,29.0,21.0,8.0,5.0,5.0,5.0,4.0,16.0
아모레퍼시픽,아모레퍼시픽우,25,29.0,19.0,30.0,17.0,12.0,6.0,2.0,2.0,2.0,9.0
LG전자,LG전자우,10,26.0,19.0,5.0,2.0,6.0,7.0,22.0,22.0,28.0,20.0
S-Oil,S-Oil우,60,17.0,31.0,18.0,13.0,9.0,8.0,6.0,6.0,30.0,23.0
아모레G,아모레G우,15,18.0,7.0,31.0,29.0,15.0,9.0,10.0,10.0,7.0,28.0
LG,LG우,20,27.0,24.0,12.0,22.0,7.0,10.0,15.0,15.0,19.0,26.0


In [333]:
df_rank.sort_values('PNL cumsum').index

MultiIndex([( 'JW중외제약',  'JW중외제약우'),
            ( '아모레퍼시픽',  '아모레퍼시픽우'),
            ( 'SK네트웍스',  'SK네트웍스우'),
            ( 'LG생활건강',  'LG생활건강우'),
            (   '현대건설',    '현대건설우'),
            (   '유한양행',    '유한양행우'),
            (   '아모레G',    '아모레G우'),
            (   'LG화학',    'LG화학우'),
            ( '한국금융지주',  '한국금융지주우'),
            (  'DB하이텍',  'DB하이텍1우'),
            (    '현대차',     '현대차우'),
            (   '삼성전자',    '삼성전자우'),
            (     '한화',      '한화우'),
            (  '넥센타이어', '넥센타이어1우B'),
            ( '미래에셋증권',  '미래에셋증권우'),
            (   '삼성화재',    '삼성화재우'),
            (     'CJ',      'CJ우'),
            (   '대한항공',    '대한항공우'),
            (     'LG',      'LG우'),
            ('SK이노베이션', 'SK이노베이션우'),
            ( 'NH투자증권',  'NH투자증권우'),
            (   '호텔신라',    '호텔신라우'),
            (  '삼성SDI',   '삼성SDI우'),
            ( 'CJ제일제당', 'CJ제일제당 우'),
            (     '대상',      '대상우'),
            (     'GS',      'GS우'),
            (  '한화솔루션',   '한화솔루션우'),
 

In [343]:
np.array(inventory_history)

array([1.00000000e+08, 1.06450986e+08, 9.79059101e+07, 1.06187881e+08,
       1.03557728e+08, 1.15669569e+08, 1.11438094e+08, 7.85882841e+07,
       7.68965726e+07, 8.01357826e+07, 8.26729069e+07, 9.21340952e+07,
       8.87681460e+07, 1.01132202e+08, 1.06905274e+08])

In [427]:
date_list = ["20090105","20110105","20130103","20150105","20170103","20190103","20210105"]
dfdf = pd.DataFrame(index=preferreds)
for d in date_list:
    mc = stock.get_market_cap(d)
    우선주시가총액 = np.array(mc['시가총액'].loc[preferred_tickers])
    dfdf[f'시가총액 {d}'] = 우선주시가총액

In [428]:
dfdf.rank(method='min', ascending=False).sort_values('시가총액 20090105')

,시가총액 20090105,시가총액 20110105,시가총액 20130103,시가총액 20150105,시가총액 20170103,시가총액 20190103,시가총액 20210105
삼성전자우,1.0,1.0,1.0,1.0,1.0,1.0,1.0
LG전자우,2.0,4.0,6.0,7.0,7.0,7.0,5.0
현대차우,3.0,2.0,2.0,2.0,2.0,2.0,3.0
LG화학우,4.0,3.0,3.0,4.0,4.0,3.0,2.0
삼성화재우,5.0,6.0,7.0,6.0,6.0,6.0,8.0
S-Oil우,6.0,7.0,8.0,13.0,10.0,8.0,16.0
아모레퍼시픽우,7.0,5.0,4.0,3.0,3.0,5.0,6.0
NH투자증권우,8.0,9.0,10.0,16.0,14.0,12.0,15.0
LG생활건강우,9.0,8.0,5.0,5.0,5.0,4.0,4.0
미래에셋증권우,10.0,11.0,17.0,18.0,18.0,20.0,21.0
